In [ ]:
!pip install -U ultralytics comet_ml "ray[tune]"

In [ ]:
import numpy as np
import urllib.request
import os
import pandas as pd
import matplotlib.pyplot as plt
import comet_ml
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
from google.colab import drive
from ultralytics import YOLO
from IPython.display import display, Image

In [ ]:
experiment = Experiment(
  api_key="5fCsrnPuH0lfw0o6xcWFmPiPL",
  project_name="general",
  workspace="xenosians"
)

In [ ]:
drive.mount('/content/gdrive')
# !ls '/content/gdrive/My Drive/'

In [ ]:
DATA_DIR = '/content/gdrive/My Drive/TA_DATA/'

In [ ]:
config_url = "https://raw.githubusercontent.com/Xenosians/ultralytics-FinalProject/main/ultralytics/cfg/models/v8/yolov8-cls.yaml"

config_path = "/content/yolov8-cls.yaml"

urllib.request.urlretrieve(config_url, config_path)

print(f"Successfully downloaded YOLOv8 model configuration file to: {config_path}")

In [ ]:
hyper_params = {
  "data" : DATA_DIR,
  "lrf": 0.001,
  "epochs": 100,
  "batch": 16,
  "pretrained" : False
}

In [ ]:
model = YOLO(config_path)
print(model)

In [ ]:
results_train = model.train(data=DATA_DIR,pretrained=False,lrf=0.001,epochs=100,batch=50,label_smoothing=1.0)

print("Training Results:")
print(results_train)

In [ ]:
experiment.log_parameters(hyper_params)

In [ ]:
!scp -r /content/runs '/content/gdrive/My Drive/TA_DATA/ImageClassifications'

In [ ]:
DATA_DIR_PREDICT_CLEAN = '/content/gdrive/MyDrive/TA_DATA/train/clean'

model = YOLO('/content/gdrive/MyDrive/TA_DATA/ImageClassifications/runs/classify/train2/weights/best.pt')

all_names = []
all_probs = []

for filename in os.listdir(DATA_DIR_PREDICT_CLEAN):
    if filename.endswith(".jpg"):
        image_path = os.path.join(DATA_DIR_PREDICT_CLEAN, filename)

        results = model(image_path)

        names_dict = results[0].names
        probs = results[0].probs.data.tolist()

        all_names.append(names_dict[np.argmax(probs)])
        all_probs.append(np.max(probs))

average_prob = np.mean(all_probs)

print("Average Result:")
print(f"Predicted Class: {all_names[np.argmax(all_probs)]}")
print(f"Average Confidence: {average_prob}")

In [ ]:
DATA_DIR_PREDICT_CHEATS = '/content/gdrive/MyDrive/TA_DATA/train/cheat/images/'

model = YOLO('/content/gdrive/MyDrive/TA_DATA/ImageClassifications/runs/classify/train2/weights/best.pt')

all_names = []
all_probs = []

for filename in os.listdir(DATA_DIR_PREDICT_CHEATS):
    if filename.endswith(".jpg"):
        image_path = os.path.join(DATA_DIR_PREDICT_CHEATS, filename)

        results = model(image_path)

        names_dict = results[0].names
        probs = results[0].probs.data.tolist()

        all_names.append(names_dict[np.argmax(probs)])
        all_probs.append(np.max(probs))

average_prob = np.mean(all_probs)

print("Average Result:")
print(f"Predicted Class: {all_names[np.argmax(all_probs)]}")
print(f"Average Confidence: {average_prob}")

In [ ]:
DATA_DIR = '/content/gdrive/MyDrive/predict/predict_random_3.jpg'

model = YOLO('/content/gdrive/MyDrive/TA_DATA/ImageClassifications/runs/classify/train2/weights/best.pt')
results = model(DATA_DIR)

image_path = DATA_DIR
display(Image(filename=image_path))

names_dict = results[0].names
probs = results[0].probs.data.tolist()

print(names_dict)
print(probs)

max_prob_index = np.argmax(probs)
predicted_class = names_dict[max_prob_index]

print(f"Predicted Class: {predicted_class}")

In [ ]:
results_path = '/content/gdrive/MyDrive/TA_DATA/ImageClassifications/runs/classify/train2/results.csv'

results = pd.read_csv(results_path)

plt.figure()
plt.plot(results['                  epoch'], results['             train/loss'], label='train loss')
plt.plot(results['                  epoch'], results['               val/loss'], label='val loss', c='red')
plt.grid()
plt.title('Loss vs epochs')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend()


plt.figure()
plt.plot(results['                  epoch'], results['  metrics/accuracy_top1'] * 100)
plt.grid()
plt.title('Validation accuracy vs epochs')
plt.ylabel('accuracy (%)')
plt.xlabel('epochs')

plt.show()

In [ ]:
Image("/content/gdrive/MyDrive/TA_DATA/ImageClassifications/runs/classify/train2/results.png")

In [ ]:
Image('/content/gdrive/MyDrive/TA_DATA/ImageClassifications/runs/classify/train2/confusion_matrix_normalized.png')

In [ ]:
model = YOLO('/content/gdrive/MyDrive/TA_DATA/ImageClassifications/runs/classify/train2/weights/best.pt')

model.export(format='onnx')